In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor langchain-chroma langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.6/364.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.1 MB/s eta 0:0

In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

# Langsmith

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]="key"
#"key"
os.environ["LANGCHAIN_PROJECT"]="Sleep 4o"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"

In [ ]:
# key

In [ ]:
print("✅ Tracing enabled:", os.environ.get("LANGCHAIN_TRACING_V2"))


✅ Tracing enabled: true


# Sleep_coach_master.py


In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from tools.spotify_tools import search_spotify_playlists
from config.setting import settings
import json
import re
import asyncio
from typing import Dict, Any
from database.mongo_client import get_database, init_database
from models.sleep_analysis import SleepAnalysis, SleepAnalysisRepository, SleepAssessment, RoutineRecommendation, SleepTips, SleepRecommendations, DisorderCheck, SleepFollowUp, SleepMetadata

class SleepCoachAgent:
    def __init__(self):
        self.model = ChatOpenAI(api_key=settings.OPENAI_API_KEY, model="gpt-4o-mini", max_tokens=10000)
        self.agent = create_react_agent(
            model=self.model,
            tools=[search_spotify_playlists],
            prompt=(
                """
                [SYSTEM MESSAGE]
                You are a JSON-only response AI. Always respond with valid JSON and nothing else. Never include explanatory text, markdown, or code blocks.

                You are MindFuel's AI sleep coach expert. Analyze the user's input text and provide a comprehensive sleep assessment in JSON format.

                For spotify recommendations, you will use the search_spotify_playlists tool strictly for the "playlist" field.

                ANALYSIS REQUIREMENTS:
                1. Identify the user's sleep issue, need, or question
                2. Extract important sleep data (time, feelings, habits, metrics)
                3. Provide a brief but insightful summary of the user's sleep pattern
                4. Detect mood and emotional cues if present
                5. Generate a personalized wind-down routine
                6. Recommend optimal sleep and wake-up timing
                7. Suggest behavior/environment improvements
                8. Perform a basic sleep disorder screening
                9. Propose follow-up actions and metrics to track
                10. Tag context categories (e.g., stress, diet, tech use)

                IMPORTANT: Respond ONLY with valid JSON. Do not include any explanatory text, markdown formatting, or code blocks. Start your response directly with the opening curly brace {{ and end with the closing curly brace }}.

                RESPONSE FORMAT (JSON):
                {{
                  "sleepAssessment": {{
                    "issue": "string",  // e.g. 'difficulty falling asleep'
                    "confidence": number,  // 0.0 to 1.0
                    "severity": "low" | "medium" | "high",
                    "summary": "string",
                    "sleepHistoryDetected": boolean,
                    "userMood": "tired" | "refreshed" | "anxious" | "groggy" | "unknown",
                    "sleepDurationTrend": "increasing" | "decreasing" | "stable" | "unknown"
                  }},
                  "routineRecommendation": {{
                    "windDown": ["string"],  // calming activities
                    "avoidBeforeBed": ["string"],  // caffeine, screens, etc.
                    "optimalSleepTime": "string",  // e.g. "10:45 PM"
                    "optimalWakeTime": "string",  // e.g. "6:30 AM"
                    "reminders": ["string"]
                  }},
                  "tips": {{
                    "immediateActions": ["string"],  // changes starting tonight
                    "lifestyleChanges": ["string"],  // longer-term habits
                    "environmentSuggestions": ["string"]  // bedroom, lighting, etc.
                  }},
                  "recommendations": {{
                      "immediate": ["string"], // 3-4 immediate actionable suggestions
                      "content": {{
                      "spotify": {{
                          "genres": ["string"], // ambient, classical, nature sounds, etc.
                          "energy": number, // 0-1 (0=calm, 1=energetic)
                          "valence": number, // 0-1 (0=sad, 1=happy)
                          "mood": "string" // relaxing, uplifting, focus, etc.
                          "playlist": [{{ // use search_spotify_playlists tool to get the playlist, IMPORTANT: do only one tool call strictly for this field, also directly use the tool call response for this field, do not parse the response yourself
                            "name": "string",
                            "description": "string",
                            "external_url": "string (URL)",
                            "image": "string (URL)",
                            "owner": {{
                                "name": "string",
                                "url": "string (URL)"
                            }},
                            "tracks": {{
                                "url": "string (API endpoint URL)",
                                "total": "integer"
                            }}
                          }}]
                      }},
                      }}
                  }},
                  "disorderCheck": {{
                    "riskLevel": "low" | "medium" | "high",
                    "symptoms": ["string"],
                    "recommendations": ["string"],
                    "complianceRisk": "low" | "medium" | "high"  // likelihood user may not follow
                  }},
                  "followUp": {{
                    "checkInPeriod": "daily" | "weekly" | "custom",
                    "trackMetrics": ["string"],  // e.g. "hours slept", "interruptions"
                    "goals": ["string"]  // e.g. "reduce latency to < 20 minutes"
                  }},
                  "metadata": {{
                    "wordCount": number,
                    "timeOfDayMentioned": "morning" | "afternoon" | "evening" | "night" | "unknown",
                    "contextTags": ["string"]  // e.g. ["stress", "caffeine", "screen use"]
                  }}
                }}

                GUIDELINES:
                - Be empathetic and non-judgmental
                - Focus on actionable insights
                - Consider cultural sensitivity
                - If serious sleep health concerns are detected, prioritize professional help recommendations
                - Tailor content suggestions to the specific sleep needs identified
                - Use clear, supportive language
                - Avoid medical diagnosis or treatment advice
                - Encourage professional help when appropriate

                [SYSTEM RULES]
                1. Respond **ONLY** with valid JSON that starts with `{{` and ends with `}}`.
                2. **Never** include:
                - Text before `{{` or after `}}`
                - Markdown/code blocks (```json```)
                - Explanatory comments
                3. If you violate this, the API will reject your response.

                [EXAMPLE OF CORRECT RESPONSE]
                {{
                    "sleepAssessment": {{
                        "issue": "Difficulty falling asleep",
                        "confidence": 0.8
                    }}
                }}

                [EXAMPLE OF INCORRECT RESPONSE]
                USER INPUT: "{input}"\n
                {{
                    "sleepAssessment": {{
                        "issue": "Difficulty falling asleep"
                    }}
                }}

                Return only the JSON object with no additional text or formatting.
                """
            ),
            name="sleep_coach_agent"
        )
        self.collection = get_collection("sleep_coach")

    def _parse_ai_response(self, result: str) -> Dict[str, Any]:
        try:
            try:
                parsed_result = json.loads(result.strip())
            except json.JSONDecodeError:
                json_str = re.search(r"\{[\s\S]*\}", result).group(0)
                parsed_result = json.loads(json_str)
            return parsed_result
        except Exception as e:
            print(f"JSON parsing error: {e}")
            raise

    def _create_sleep_analysis_model(self, parsed_result: Dict[str, Any], user_id: str, user_input: str, context: dict) -> SleepAnalysis:
        try:
            sleep_assessment_data = parsed_result.get("sleepAssessment", {})
            routine_data = parsed_result.get("routineRecommendation", {})
            tips_data = parsed_result.get("tips", {})
            recommendations_data = parsed_result.get("recommendations", {})
            disorder_data = parsed_result.get("disorderCheck", {})
            followup_data = parsed_result.get("followUp", {})
            metadata_data = parsed_result.get("metadata", {})

            sleep_assessment = SleepAssessment(
                issue=sleep_assessment_data.get("issue", "unknown"),
                confidence=sleep_assessment_data.get("confidence", 0.5),
                severity=sleep_assessment_data.get("severity", "low"),
                summary=sleep_assessment_data.get("summary", ""),
                sleepHistoryDetected=sleep_assessment_data.get("sleepHistoryDetected", False),
                userMood=sleep_assessment_data.get("userMood", "unknown"),
                sleepDurationTrend=sleep_assessment_data.get("sleepDurationTrend", "unknown")
            )

            routine_recommendation = RoutineRecommendation(
                windDown=routine_data.get("windDown", []),
                avoidBeforeBed=routine_data.get("avoidBeforeBed", []),
                optimalSleepTime=routine_data.get("optimalSleepTime", ""),
                optimalWakeTime=routine_data.get("optimalWakeTime", ""),
                reminders=routine_data.get("reminders", [])
            )

            tips = SleepTips(
                immediateActions=tips_data.get("immediateActions", []),
                lifestyleChanges=tips_data.get("lifestyleChanges", []),
                environmentSuggestions=tips_data.get("environmentSuggestions", [])
            )

            content_data = recommendations_data.get("content", {})
            recommendations = SleepRecommendations(
                immediate=recommendations_data.get("immediate", []),
                content={
                    "spotify": {
                        "genres": content_data.get("spotify", {}).get("genres", []),
                        "energy": content_data.get("spotify", {}).get("energy", 0.5),
                        "valence": content_data.get("spotify", {}).get("valence", 0.5),
                        "mood": content_data.get("spotify", {}).get("mood", "relaxing"),
                        "playlist": content_data.get("spotify", {}).get("playlist", None)
                    }
                }
            )

            disorder_check = DisorderCheck(
                riskLevel=disorder_data.get("riskLevel", "low"),
                symptoms=disorder_data.get("symptoms", []),
                recommendations=disorder_data.get("recommendations", []),
                complianceRisk=disorder_data.get("complianceRisk", "medium")
            )

            followup = SleepFollowUp(
                checkInPeriod=followup_data.get("checkInPeriod", "weekly"),
                trackMetrics=followup_data.get("trackMetrics", []),
                goals=followup_data.get("goals", [])
            )

            metadata = SleepMetadata(
                wordCount=metadata_data.get("wordCount", 0),
                timeOfDayMentioned=metadata_data.get("timeOfDayMentioned", "unknown"),
                contextTags=metadata_data.get("contextTags", [])
            )

            return SleepAnalysis(
                userId=user_id,
                input=user_input,
                context=context,
                sleepAssessment=sleep_assessment,
                routineRecommendation=routine_recommendation,
                tips=tips,
                recommendations=recommendations,
                disorderCheck=disorder_check,
                followUp=followup,
                metadata=metadata
            )

        except Exception as e:
            print(f"Error creating SleepAnalysis model: {e}")
            raise

    async def run(self, user_input: str, user_id: str = None, context: dict = None, save_to_db: bool = True) -> Dict[str, Any]:
        try:
            prompt = (
                f"""
                CONTEXT:
                    - MindFuel is a mental wellness app that helps users track sleep, get personalized content recommendations, and improve sleep health
                    - Users share their sleep patterns, issues, and current state
                    - Your analysis will be used to provide personalized Spotify playlists and sleep recommendations
                    - Be empathetic, supportive, and professional in your analysis

                USER INPUT:
                {user_input}
                """
            )

            response = await asyncio.to_thread(self.agent.invoke, {
                "input": prompt
            })

            content = response.get("output", "")

            if isinstance(content, dict):
                parsed_result = content  # already a dict
            elif isinstance(content, str):
                try:
                    parsed_result = json.loads(content)
                except json.JSONDecodeError as e:
                    return {"error": f"Failed to parse JSON: {str(e)}"}
            else:
                return {"error": "Unknown content format from sleep coach."}

            db_info = {}
            full_analysis_model = self._create_sleep_analysis_model(parsed_result, user_id, user_input, context)

            if save_to_db and user_id:
                try:
                    await init_database()

                    full_repo = SleepAnalysisRepository()
                    full_analysis_id = await full_repo.create(full_analysis_model)

                    db_info = {
                        "full_analysis_id": full_analysis_id,
                        "saved": True
                    }
                except Exception as e:
                    print(f"Error saving to database: {e}")
                    db_info = {"saved": False, "error": str(e)}

            return {
                "analysis": parsed_result,
                "frontend_format": full_analysis_model.from_mongo(full_analysis_model.to_mongo()),
                "database": db_info
            }

        except Exception as e:
            print(f"Run error: {e}")
            return {"error": str(e)}

    async def run3(self, user_input: str, user_id: str = None, context: str = None, save_to_db: bool = True) -> Dict[str, Any]:
        try:
            context_str = """
                - MindFuel is a mental wellness app that helps users track sleep, get personalized content recommendations, and improve sleep health
                - Users share their sleep patterns, issues, and current state
                - Your analysis will be used to provide personalized Spotify playlists and sleep recommendations
                - Be empathetic, supportive, and professional in your analysis
            """

            prompt_vars = {
                "context": context_str,
                "input": user_input,
            }

            result = await asyncio.to_thread(self.chain.invoke, prompt_vars)
            parsed_result = self._parse_ai_response(result)

            try:
                from agents import get_agent

                spotify_rec = parsed_result.get("recommendations", {}).get("content", {}).get("spotify")

                tasks = []
                if spotify_rec:
                    spotify_agent = get_agent("content_generator_agent")
                    tasks.append(asyncio.to_thread(spotify_agent.run, "spotify", spotify_rec))

                if tasks:
                    recommendation_results = await asyncio.gather(*tasks)

                    if spotify_rec:
                        parsed_result["recommendations"]["content"]["spotify"]["playlist"] = recommendation_results.pop(0)

            except Exception as e:
                print(f"Error getting content recommendations: {e}")

            db_info = {}
            full_analysis_model = self._create_sleep_analysis_model(parsed_result, user_id, user_input, context)

            if save_to_db and user_id:
                try:
                    await init_database()

                    full_repo = SleepAnalysisRepository()
                    full_analysis_id = await full_repo.create(full_analysis_model)

                    db_info = {
                        "full_analysis_id": full_analysis_id,
                        "saved": True
                    }
                except Exception as e:
                    print(f"Error saving to database: {e}")
                    db_info = {"saved": False, "error": str(e)}

            return {
                "analysis": parsed_result,
                "frontend_format": full_analysis_model.from_mongo(full_analysis_model.to_mongo()),
                "database": db_info
            }

        except Exception as e:
            print(f"Run error: {e}")
            return {"error": str(e)}

    def run2(self, user_input, user_id=None, context=None):
        try:
            context_str = """
                - MindFuel is a mental wellness app that helps users track sleep, get personalized content recommendations, and improve sleep health
                - Users share their sleep patterns, issues, and current state
                - Your analysis will be used to provide personalized Spotify playlists and sleep recommendations
                - Be empathetic, supportive, and professional in your analysis
            """
            prompt_vars = {
                "context": context_str,
                "input": user_input,
            }
            return self.chain.invoke(prompt_vars)
        except Exception as e:
            print(e)
            return {"error": str(e)}

    async def get_user_analyses(self, user_id: str, limit: int = 10, simple: bool = True) -> Dict[str, Any]:
        try:
            await init_database()

            repo = SleepAnalysisRepository()
            analyses = await repo.get_by_user_id(user_id, limit)

            if simple:
                return {
                    "analyses": [analysis.to_simple_format() for analysis in analyses],
                    "type": "simple"
                }
            else:
                return {
                    "analyses": [analysis.dict() for analysis in analyses],
                    "type": "full"
                }
        except Exception as e:
            print(f"Error getting user analyses: {e}")
            return {"error": str(e)}

    async def get_sleep_trends(self, user_id: str, days: int = 30) -> Dict[str, Any]:
        try:
            await init_database()
            db = await get_database()
            repo = SleepAnalysisRepository(db)
            trends = await repo.get_user_sleep_trends(user_id, days)
            return {"trends": trends}
        except Exception as e:
            print(f"Error getting sleep trends: {e}")
            return {"error": str(e)}

# sleep_analysis.py

In [ ]:
from datetime import datetime, timedelta
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field
from bson import ObjectId
from database.mongo_client import get_database


class SleepAssessment(BaseModel):
    issue: str
    confidence: float = Field(ge=0, le=1)
    severity: str  # low, medium, high
    summary: str
    sleepHistoryDetected: bool
    userMood: str  # tired, refreshed, anxious, groggy, unknown
    sleepDurationTrend: str  # increasing, decreasing, stable, unknown


class RoutineRecommendation(BaseModel):
    windDown: List[str]
    avoidBeforeBed: List[str]
    optimalSleepTime: str
    optimalWakeTime: str
    reminders: List[str]


class SleepTips(BaseModel):
    immediateActions: List[str]
    lifestyleChanges: List[str]
    environmentSuggestions: List[str]


class SpotifyPlaylist(BaseModel):
    name: str
    description: str
    external_url: str
    image: str
    owner: Dict[str, str]
    tracks: Dict[str, Any]


class SleepContentRecommendations(BaseModel):
    spotify: Dict[str, Any]  # Contains spotify playlist recommendations


class SleepRecommendations(BaseModel):
    immediate: List[str]
    content: SleepContentRecommendations


class DisorderCheck(BaseModel):
    riskLevel: str  # low, medium, high
    symptoms: List[str]
    recommendations: List[str]
    complianceRisk: str  # low, medium, high


class SleepFollowUp(BaseModel):
    checkInPeriod: str  # daily, weekly, custom
    trackMetrics: List[str]
    goals: List[str]


class SleepMetadata(BaseModel):
    wordCount: int
    timeOfDayMentioned: str  # morning, afternoon, evening, night, unknown
    contextTags: List[str]  # e.g., ["stress", "caffeine", "screen use"]


class SleepAnalysis(BaseModel):
    id: Optional[str] = Field(default=None, alias="_id")
    userId: str = Field(alias="userId")
    input: str = ""
    context: dict = {}
    sleepAssessment: SleepAssessment
    routineRecommendation: RoutineRecommendation
    tips: SleepTips
    recommendations: SleepRecommendations
    disorderCheck: DisorderCheck
    followUp: SleepFollowUp
    metadata: SleepMetadata
    createdAt: datetime = Field(default_factory=datetime.utcnow)
    updatedAt: datetime = Field(default_factory=datetime.utcnow)

    class Config:
        validate_by_name = True
        json_encoders = {
            ObjectId: str,
            datetime: lambda v: v.isoformat()
        }

    def to_mongo(self) -> Dict[str, Any]:
        """Convert to MongoDB document format"""
        data = self.dict(exclude={"id"})
        if self.id:
            data["_id"] = ObjectId(self.id)
        data["userId"] = ObjectId(data["userId"])
        return data

    @classmethod
    def from_mongo(cls, data: Dict[str, Any]) -> "SleepAnalysis":
        """Create instance from MongoDB document"""
        if data is None:
            return None

        # Create a copy to avoid modifying the original data
        data_copy = data.copy()
        if data_copy.get("_id"):
            data_copy["id"] = str(data_copy["_id"])
            del data_copy["_id"]  # Remove the ObjectId field
        if data_copy.get("userId"):
            data_copy["userId"] = str(data_copy["userId"])
        return cls(**data_copy)

    def to_simple_format(self) -> Dict[str, Any]:
        return {
            "id": self.id,
            "userId": self.userId,
            "issue": self.sleepAssessment.issue,
            "severity": self.sleepAssessment.severity,
            "summary": self.sleepAssessment.summary,
            "recommendations": self.recommendations.immediate[:3],
            "createdAt": self.createdAt.isoformat()
        }


class SleepAnalysisRepository:
    def __init__(self, db=None):
        self._db = db
        self._collection = None

    @property
    async def collection(self):
        if self._collection is None:
            if self._db is None:
                db = await get_database()
            else:
                db = self._db
            self._collection = db.sleep_analyses
        return self._collection

    async def create(self, sleep_analysis: SleepAnalysis) -> str:
        doc = sleep_analysis.to_mongo()
        result = await (await self.collection).insert_one(doc)
        return str(result.inserted_id)

    async def get_by_id(self, analysis_id: str) -> Optional[SleepAnalysis]:
        doc = await (await self.collection).find_one({"_id": ObjectId(analysis_id)})
        return SleepAnalysis.from_mongo(doc) if doc else None

    async def get_by_user_id(self, user_id: str, limit: int = 10) -> List[SleepAnalysis]:
        cursor = (await self.collection).find({"userId": ObjectId(user_id)}).sort("createdAt", -1).limit(limit)
        docs = await cursor.to_list(length=limit)
        return [SleepAnalysis.from_mongo(doc) for doc in docs]

    async def update(self, analysis_id: str, updates: Dict[str, Any]) -> bool:
        updates["updatedAt"] = datetime.utcnow()
        result = await (await self.collection).update_one(
            {"_id": ObjectId(analysis_id)},
            {"$set": updates}
        )
        return result.modified_count > 0

    async def delete(self, analysis_id: str) -> bool:
        result = await (await self.collection).delete_one({"_id": ObjectId(analysis_id)})
        return result.deleted_count > 0

    async def get_recent_analyses(self, limit: int = 20) -> List[SleepAnalysis]:
        cursor = (await self.collection).find().sort("createdAt", -1).limit(limit)
        docs = await cursor.to_list(length=limit)
        return [SleepAnalysis.from_mongo(doc) for doc in docs]

    async def get_analyses_by_issue(self, issue: str, limit: int = 10) -> List[SleepAnalysis]:
        cursor = (await self.collection).find({"sleepAssessment.issue": issue}).sort("createdAt", -1).limit(limit)
        docs = await cursor.to_list(length=limit)
        return [SleepAnalysis.from_mongo(doc) for doc in docs]

    async def get_user_sleep_trends(self, user_id: str, days: int = 30) -> List[Dict[str, Any]]:
        pipeline = [
            {"$match": {"userId": ObjectId(user_id)}},
            {"$match": {"createdAt": {"$gte": datetime.utcnow() - timedelta(days=days)}}},
            {"$group": {
                "_id": "$sleepAssessment.issue",
                "count": {"$sum": 1},
                "avgConfidence": {"$avg": "$sleepAssessment.confidence"},
                "avgSeverity": {"$avg": {
                    "$switch": {
                        "branches": [
                            {"case": {"$eq": ["$sleepAssessment.severity", "low"]}, "then": 1},
                            {"case": {"$eq": ["$sleepAssessment.severity", "medium"]}, "then": 2},
                            {"case": {"$eq": ["$sleepAssessment.severity", "high"]}, "then": 3}
                        ],
                        "default": 0
                    }
                }}
            }},
            {"$sort": {"count": -1}}
        ]
        return await (await self.collection).aggregate(pipeline).to_list(None)

    async def get_analyses_by_date_range(self, user_id: str, start_date: datetime, end_date: datetime) -> List[SleepAnalysis]:
        cursor = (await self.collection).find({
            "userId": ObjectId(user_id),
            "createdAt": {"$gte": start_date, "$lte": end_date}
        }).sort("createdAt", -1)
        docs = await cursor.to_list(None)
        return [SleepAnalysis.from_mongo(doc) for doc in docs]

    async def get_high_risk_analyses(self, user_id: str) -> List[SleepAnalysis]:
        cursor = (await self.collection).find({
            "userId": ObjectId(user_id),
            "disorderCheck.riskLevel": {"$in": ["high", "medium"]}
        }).sort("createdAt", -1)
        docs = await cursor.to_list(None)
        return [SleepAnalysis.from_mongo(doc) for doc in docs]

    async def get_analyses_by_time_of_day(self, user_id: str, time_of_day: str) -> List[SleepAnalysis]:
        cursor = (await self.collection).find({
            "userId": ObjectId(user_id),
            "metadata.timeOfDayMentioned": time_of_day
        }).sort("createdAt", -1)
        docs = await cursor.to_list(None)
        return [SleepAnalysis.from_mongo(doc) for doc in docs]

    async def get_analyses_by_context_tag(self, user_id: str, tag: str) -> List[SleepAnalysis]:
        cursor = (await self.collection).find({
            "userId": ObjectId(user_id),
            "metadata.contextTags": tag
        }).sort("createdAt", -1)
        docs = await cursor.to_list(None)
        return [SleepAnalysis.from_mongo(doc) for doc in docs]